In [1]:
from bs4 import BeautifulSoup
import requests
import smtplib
import time
import os
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import mysql.connector
db_password = os.getenv('DB_PASSWORD')
mydb = mysql.connector.connect(host = "localhost", user = "root", password = db_password, database = "job_details")
cur = mydb.cursor()

In [2]:
print('Enter skills, designation, keyword to filter: ')
skillset = input("")
#skillset = ["java", "python", "c++", "data science", "machine learning", "data engineer", "full stack", "web development", "software engineer"]
#java, python, c++, data science, machine learning, data engineer, full stack, web development, software engineer
skillset = skillset.split(",")
skillset = [item.strip() for item in skillset]
size = len(skillset)
url = "https://www.timesjobs.com/candidate/job-search.html?searchType=personalizedSearch&from=submit&searchTextSrc=&searchTextText=&txtKeywords="
end = "&txtLocation="
for i in range(0, size):
    if skillset[i] == 'c++' or skillset[i] == 'C++':
        skillset[i] = "c%2B%2B"
if size == 1:
    skillset = str(skillset[0]).replace(' ', '+')
    url = url + skillset + end
else:
    for val in range(0, size - 1):    
        skillset[val] = (skillset[val] + "%2C+").replace(' ', '+')
        url += skillset[val]

url += str(skillset[size - 1]).replace(' ', '+') + end
print(url)

Enter skills, designation, keyword to filter: 
java, python, c++, data science, machine learning, data engineer, full stack, web development, software engineer
https://www.timesjobs.com/candidate/job-search.html?searchType=personalizedSearch&from=submit&searchTextSrc=&searchTextText=&txtKeywords=java%2C+python%2C+c%2B%2B%2C+data+science%2C+machine+learning%2C+data+engineer%2C+full+stack%2C+web+development%2C+software+engineer&txtLocation=


In [3]:
html_file = requests.get(url).text

In [4]:
soup = BeautifulSoup(html_file, 'lxml')
jobs = soup.find_all('li', class_ = 'clearfix job-bx wht-shd-bx')

In [5]:
cur.execute("SELECT companyName, skills, url FROM details")
existing_data = cur.fetchall()
existing_data_set = set((x[0], x[1], x[2]) for x in existing_data)

In [6]:
def fetch_and_send():
    new_jobs = []
    for job in jobs:
        published_date = job.find('span', class_ = 'sim-posted').text
        if 'day' or 'few' in published_date:
            company_name = job.find('h3', class_ = 'joblist-comp-name').text.replace(' ', '')
            skills = job.find('span', class_ = 'srp-skills').text.replace(' ', '')
            more_info = job.header.a['href']
            db_values = (company_name, skills, more_info)
        
            if db_values not in existing_data_set:
                cur.execute("INSERT INTO details (companyName, skills, url) values (%s, %s, %s)", db_values)
                mydb.commit()
                existing_data_set.add(db_values)
                new_jobs.append(db_values)
            print(company_name.strip())
            print(skills.strip())
            print(more_info.strip())
            print("")
    if new_jobs:
        send_email(new_jobs)

In [7]:
def send_email(new_jobs):
    email_password = os.getenv('EMAIL_PASSWORD')
    from_email = "itsstudentdemomail@gmail.com"
    #to_emails = ["21031a0523@gmail.com", "veerendrababu80@gmail.com", "duddiajay119@gmail.com", "gangadharpalla11@gmail.com", "mounikaamgothu2001@gmail.com", "vhemanthsai68@gmail.com", "21031a0451@gmail.com", "hariprasadnaikamgothu@gmail.com"]
    to_emails = ["hariprasadnaikamgothu@gmail.com", "21031a0523@gmail.com"]
    subject = "New Job Posting"
    body = f"""
    <html>
    <head></head>
    <body>
        <h1>New Job Posting</h1>
        <table border="1" cellpadding="10" cellspacing="0">
         <tr>
             <th>Company</th>
             <th>Skills</th>
             <th>URL</th>
        </tr>
    """
    for job in new_jobs:
        company_name, skills, url = job;
        body += f"""
        <tr>
            <td>{company_name}</td>
            <td>{skills}</td>
            <td><a href="{url}">View Job Details</a></td>
        </tr>
    """
    body += f"""
            </table>
            </body>
            </html>
         """
    for to_email in to_emails:
        msg = MIMEMultipart()
        msg['From'] = from_email
        msg['To'] = to_email
        msg['Subject'] = subject
        msg.attach(MIMEText(body, 'html'))
        
        try:
            with smtplib.SMTP('smtp.gmail.com', 587) as server:
                server.starttls()
                server.login(from_email, email_password)
                server.sendmail(from_email, to_email, msg.as_string())
                print(f'Email sent to {to_email}')
        except Exception as e:
            print(f'Error: {e}')

In [ ]:
if __name__ == '__main__':
    while True:
        fetch_and_send()
        time_wait = 60
        print(f'waiting {time_wait} mins...')
        time.sleep(time_wait * 60)

AnalyticsVidhya
hive,python,machinelearning,pig,neo4j,cassandra,hadoop,bigdata
https://www.timesjobs.com/job-detail/data-science-machine-learning-engineer-analytics-vidhya-mumbai-4-to-5-yrs-jobid-6xN8SfmAZqVzpSvf__PLUS__uAgZw==&source=srp

aitouchllp
scala,erlang,artificialintelligence,docker,spring,java,devops,mqtt,python,naturallanguageprocessing,machinelearning,javascript,nosql,node.js,cassandra
https://www.timesjobs.com/job-detail/full-stack-data-science-architect-aitouch-llp-gurgaon-4-to-7-yrs-jobid-E4__SLASH__N6vr0QlBzpSvf__PLUS__uAgZw==&source=srp

MaxgenTechnologies
(MoreJobs)
internship,datascience
https://www.timesjobs.com/job-detail/data-science-internship-in-ahmedabad-maxgen-technologies-ahmedabad-mehsana-rajkot-surat-surendranagar-0-to-1-yrs-jobid-PlyRasRVU5VzpSvf__PLUS__uAgZw==&source=srp

SAMPOORNACONSULTANTSPVTLTD
(MoreJobs)
Python,Azure,ML
https://www.timesjobs.com/job-detail/machine-learning-python-azure-am-blr-j48114-sampoorna-consultants-pvt-ltd-bengaluru-bangalore-